In [1]:
import sys
sys.path.append("..")
import torch
from utils import DATA_HOME
from lstm.sales_data import Sales_Dataset

sd = Sales_Dataset(f"{DATA_HOME}\sales_data")

In [2]:
tr, ts, h, o = sd[10]
len(tr), len(ts), len(h), len(o)

(            id        date  store_nbr                      family    sales   
 190674  190674  2013-04-18          1                  AUTOMOTIVE    0.000  \
 190675  190675  2013-04-18          1                   BABY CARE    0.000   
 190676  190676  2013-04-18          1                      BEAUTY    1.000   
 190677  190677  2013-04-18          1                   BEVERAGES  785.000   
 190678  190678  2013-04-18          1                       BOOKS    0.000   
 ...        ...         ...        ...                         ...      ...   
 192451  192451  2013-04-18          9                     POULTRY  414.885   
 192452  192452  2013-04-18          9              PREPARED FOODS   73.000   
 192453  192453  2013-04-18          9                     PRODUCE    0.000   
 192454  192454  2013-04-18          9  SCHOOL AND OFFICE SUPPLIES    0.000   
 192455  192455  2013-04-18          9                     SEAFOOD   35.752   
 
         onpromotion  
 190674            0  
 190